# Inverse Friction Evolution Using PINNs

This notebook solves the inverse friction evolution problem using Physics-Informed Neural Networks (PINNs). In this inverse problem, we aim to infer unknown friction parameters (for example, $f_0$, $a$, and $b$) in the rate-and-state friction law from measured data.

The model is based on the following nondimensionalized system of ordinary differential equations (ODEs):

\begin{align*}
\frac{d\tilde{u}}{d\tilde{t}} &= \tilde{v}, \\
\frac{d\tilde{v}}{d\tilde{t}} &= \kappa\big(v_0 \tilde{t} - \tilde{u}\big) - \alpha\Big( f_0 + a \ln \tilde{v} + b \ln \tilde{\theta} \Big), \\
\frac{d\tilde{\theta}}{d\tilde{t}} &= -\tilde{v}\tilde{\theta}\ln\big(\tilde{v}\tilde{\theta}\big).
\end{align*}

In the inverse setting the friction parameters $f_0$, $a$ and $b$ are treated as unknown and are estimated during training.

The overall loss used for training is composed of:

- **Residual loss ($\text{MSE}_R$)**: Enforcing that the network output satisfies the ODE system.
- **Boundary loss ($\text{MSE}_B$)**: Ensuring the initial conditions are met.
- **Measurement loss ($\text{MSE}_m$)**: Penalizing the difference between the network output and measured data.

Below, we detail each step in the workflow.

## Import Libraries

We begin by importing the necessary libraries: DeepXDE (for PINNs), NumPy, Matplotlib, Pandas, and the backend (TensorFlow) from DeepXDE.

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import integrate
from deepxde.backend import tf

## Dataset Variables Explanation

The dataset (CSV file) contains the following columns:

- **`Var1`**: Time (independent variable, spanning from 0 to 100 seconds).
- **`y1_1`**: $\tilde{u}$ (slip) – the displacement in the spring-block slider model.
- **`y1_2`**: $\tilde{v}$ (slip rate) – the time derivative of slip (velocity).
- **`y1_3`**: $\theta$ (state variable) – a variable from the rate-and-state friction law.

These measured values (denoted $u^*$, $v^*$, and $\theta^*$) will be used to enforce the measurement loss.

In [ ]:
# Read the dataset (taking the first 10,000 data points)
raw = pd.read_csv('./../Dataset/sbm1.csv')
raw = raw[0:10000]

# Extract the columns
observe_t = raw['Var1']
u_ext = raw['y1_1']
v_ext = raw['y1_2']
theta_ext = raw['y1_3']

## Interpolating and Plotting Measurements

We interpolate the full dataset to obtain **25 equidistant measurement points** from time 0 to 100. 
These sparse points will be used as the observed data in the PINN training.

In [ ]:
t_int = np.linspace(0, 100, 25)

u_int = np.interp(t_int, observe_t.values.reshape(-1), u_ext.values.reshape(-1))
v_int = np.interp(t_int, observe_t.values.reshape(-1), v_ext.values.reshape(-1))
theta_int = np.interp(t_int, observe_t.values.reshape(-1), theta_ext.values.reshape(-1))

plt.figure(figsize=(8,6))
plt.plot(observe_t, u_ext, label="Full Slip (u)")
plt.plot(observe_t, v_ext, label="Full Slip Rate (v)")
plt.plot(observe_t, theta_ext, label= r"Full State (\(\theta\))")

plt.scatter(t_int, u_int, color='red', label="Measured u")
plt.scatter(t_int, v_int, color='orange', label="Measured v")
plt.scatter(t_int, theta_int, color='green', label= r"Measured \(\theta\)")

plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show()

# Reshape the interpolated data for DeepXDE (each as a column vector)
observe_t = t_int.reshape((-1, 1))
u_ext = u_int.reshape((-1, 1))
v_ext = v_int.reshape((-1, 1))
theta_ext = theta_int.reshape((-1, 1))

## Defining Measurement Boundary Conditions

We use DeepXDE's `PointSetBC` to enforce that the network solution matches the measured data at the chosen time points.

Each boundary condition is defined for one of the three outputs: $\tilde{u}$, $\tilde{v}$, and $\theta$.

In [ ]:
observe_y0 = dde.icbc.PointSetBC(observe_t, u_ext, component=0)
observe_y1 = dde.icbc.PointSetBC(observe_t, v_ext, component=1)
observe_y2 = dde.icbc.PointSetBC(observe_t, theta_ext, component=2)

## Inverse Problem Formulation

In addition to solving the forward ODE system, we now treat the friction parameters $f_0$, $a$, and $b$ as unknowns that will be inferred during training.

The inverse problem uses the same ODE system as before:

\begin{align*}
\frac{d\tilde{u}}{d\tilde{t}} &= \tilde{v}, \\
\frac{d\tilde{v}}{d\tilde{t}} &= \kappa\big(v_0 \tilde{t} - \tilde{u}\big) - \alpha\Big( f_0 + a \ln \tilde{v} + b \ln \tilde{\theta} \Big), \\
\frac{d\tilde{\theta}}{d\tilde{t}} &= -\tilde{v}\tilde{\theta}\ln\big(\tilde{v}\tilde{\theta}\big).
\end{align*}

In our PINN formulation, the friction parameters are set as trainable variables (with initial guesses).

In [ ]:
def inverse_ode_system(x, y):
    """
    Define the residuals of the ODE system for the inverse friction evolution problem.
    In this formulation, the friction parameters f0, a, and b are treated as trainable variables.
    """
    # y has three columns corresponding to u, v, and theta
    u = y[:, 0:1]
    v = y[:, 1:2]
    theta = y[:, 2:3]

    # Compute time derivatives using automatic differentiation
    du_t = dde.grad.jacobian(y, x, i=0)
    dv_t = dde.grad.jacobian(y, x, i=1)
    dtheta_t = dde.grad.jacobian(y, x, i=2)

    # Define trainable friction parameters (initial guesses)
    f0_inv = dde.Variable(0.2)
    a_inv  = dde.Variable(0.2)
    b_inv  = dde.Variable(0.3)

    # Clip v and theta to avoid issues with logarithms (avoid non-positive values)
    v_clip = tf.clip_by_value(v, 1e-6, 13)
    theta_clip = tf.clip_by_value(theta, 1e-6, 11)

    # Define the residuals
    res_u = du_t - v_clip
    res_v = dv_t - (kappa * (v0 * x - u) - alpha * (f0_inv + a_inv * tf.math.log(v_clip) + b_inv * tf.math.log(theta_clip)))
    res_theta = dtheta_t + v_clip * theta_clip * tf.math.log(v_clip * theta_clip)

    return [res_u, res_v, res_theta]

## Compile and Train the PINN Model

We now set up the PINN for the inverse problem. The following steps are used:

- **Geometry**: The time domain is defined as \([0, 100]\).
- **Data**: The PINN is enforced to satisfy the inverse ODE system and to match the measured data
  (using 20,000 residual points and the 25 measurement points defined earlier).
- **Network Architecture**: A feed-forward neural network with 6 hidden layers of 64 neurons each, 
  using the hyperbolic tangent (tanh) activation function. The network has 1 input (time) and 3 outputs.
- **Output Transform**: An output transform is applied to help the network meet the initial conditions.
- **Training**: The model is compiled with the Adam optimizer (learning rate = 0.0001) and trained for 50,000 iterations.

During training the unknown friction parameters are adjusted so that the predicted dynamics match the observed data.

In [ ]:
geom = dde.geometry.TimeDomain(0, 100)

data = dde.data.PDE(
    geom,
    inverse_ode_system,
    [observe_y0, observe_y1, observe_y2],
    20000,
    0,
    num_test=3000
)

In [ ]:
layer_size = [1] + [64] * 6 + [3]
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

In [ ]:
def output_transform(t, y):
    """
    Output transform to help the network satisfy initial conditions.
    The network outputs are shifted to approximately match:
    u ~ tanh(t) * y1 + 1, v ~ tanh(t) * y2 + 0.5, theta ~ tanh(t) * y3 + 1
    """
    y1 = y[:, 0:1]
    y2 = y[:, 1:2]
    y3 = y[:, 2:3]
    return tf.concat([
        y1 * tf.tanh(t) + 1,
        y2 * tf.tanh(t) + 0.5,
        y3 * tf.tanh(t) + 1
    ], axis=1)

net.apply_output_transform(output_transform)

In [ ]:
model = dde.Model(data, net)
model.compile(
    "adam",
    lr=0.0001,
    loss_weights=[1, 1, 1, 1, 1, 1]
)

# Create output directory for saving checkpoints
path = "./../output/Model/model"
os.makedirs(path, exist_ok=True)
checkpoint_path = os.path.join(path, "model.ckpt")
checker = dde.callbacks.ModelCheckpoint(
      checkpoint_path, save_better_only=True, period=50
  )

# Train the model for 50,000 iterations (epochs is deprecated; use iterations instead)
losshistory, train_state = model.train(epochs=50000, callbacks=[checker])

## Prediction and Plotting

After training the PINN, we predict the solution over the time domain and compare the network predictions with the true measured data. 

This visualization allows us to assess how well the inverse PINN has inferred the dynamics and, indirectly, the unknown friction parameters.

In [ ]:
observe_t = raw['Var1']
u_ext = raw['y1_1']
v_ext = raw['y1_2']
theta_ext = raw['y1_3']

plt.figure(figsize=(8,6))
plt.xlabel("Time")
plt.ylabel("y")

plt.plot(observe_t, u_ext, color="black", label="True u")
plt.plot(observe_t, v_ext, color="blue", label="True v")
plt.plot(observe_t, theta_ext, color="brown", label=r'True $\theta$')

t = np.linspace(0, 100, 10000).reshape((-1, 1))
sol_pred = model.predict(t)
u_pred = sol_pred[:, 0:1]
v_pred = sol_pred[:, 1:2]
theta_pred = sol_pred[:, 2:3]

plt.plot(t, u_pred, color="red", linestyle="dashed", label="Predict u")
plt.plot(t, v_pred, color="orange", linestyle="dashed", label="Predict v")
plt.plot(t, theta_pred, color="green", linestyle="dashed", label=r"Predict $\theta$")
plt.legend()
plt.grid(True)
plt.savefig('./../output/pred_inverse.png')
plt.show()